In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('main_enron_emails.csv')

In [3]:
df.head()

,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...


In [4]:
df.tail()

,file,message
517396,zufferli-j/sent_items/95.,Message-ID: <26807948.1075842029936.JavaMail.e...
517397,zufferli-j/sent_items/96.,Message-ID: <25835861.1075842029959.JavaMail.e...
517398,zufferli-j/sent_items/97.,Message-ID: <28979867.1075842029988.JavaMail.e...
517399,zufferli-j/sent_items/98.,Message-ID: <22052556.1075842030013.JavaMail.e...
517400,zufferli-j/sent_items/99.,Message-ID: <28618979.1075842030037.JavaMail.e...


In [5]:
df.describe()

,file,message
count,517401,517401
unique,517401,517401
top,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
freq,1,1


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517401 entries, 0 to 517400
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   file     517401 non-null  object
 1   message  517401 non-null  object
dtypes: object(2)
memory usage: 7.9+ MB


In [7]:
# Drop missing messages
df.dropna(subset=["message"])

,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...
...,...,...
517396,zufferli-j/sent_items/95.,Message-ID: <26807948.1075842029936.JavaMail.e...
517397,zufferli-j/sent_items/96.,Message-ID: <25835861.1075842029959.JavaMail.e...
517398,zufferli-j/sent_items/97.,Message-ID: <28979867.1075842029988.JavaMail.e...
517399,zufferli-j/sent_items/98.,Message-ID: <22052556.1075842030013.JavaMail.e...


In [8]:
# Reset index for clarity
df = df.reset_index(drop=True)

In [9]:
pd.set_option('display.max_colwidth', None)
df.loc[1]

file                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [10]:
# Function to split the message into subject and body
def extract_subject_body(text):
    #splits each line where it sees \n
    lines = text.split("\n")
    subject = ""
    body_lines = []
    body_started = False

    for line in lines:
        if line.lower().startswith("subject:"):
            #if a line starts with subject, subject is 8 words, remove subject and save he details of that line into the subject variable
            subject = line[8:].strip()#remove "subject:"
        elif body_started:
            #when body started is set to true
            body_lines.append(line)#add each line to the body list
        elif line.strip() == "":
            body_started = True #Body starts after first empty line

    body = "\n".join(body_lines).strip()
    #wherever backslash n exists, join the lines and also remove spaces 
    return pd.Series([subject, body]) #this returns a new row with 2 columns

In [11]:
# Apply to each row, now i have successfully extracted subject and body 
df[["subject", "body"]] = df["message"].apply(extract_subject_body)

In [12]:
df[["subject", "body"]].head(3)

,subject,body
0,,Here is our forecast
1,Re:,"Traveling to have a business meeting takes the fun out of the trip. Especially if you have to prepare a presentation. I would suggest holding the business plan meetings here then take a trip without any formal business meetings. I would even try and get some honest opinions on whether a trip is even desired or necessary.\n\nAs far as the business meetings, I think it would be more productive to try and stimulate discussions across the different groups about what is working and what is not. Too often the presenter speaks and the others are quiet just waiting for their turn. The meetings might be better if held in a round table discussion format. \n\nMy suggestion for where to go is Austin. Play golf and rent a ski boat and jet ski's. Flying somewhere takes too much time."
2,Re: test,test successful. way to go!!!


In [13]:
#From the body, remove whitespaces, then convert each line of the body to bool, if it contains text true else false
#then keep only rows where the condition true
df = df[df["body"].str.strip().astype(bool)]

In [14]:
# Remove whitespaces/extra spaces from subject
df["subject"] = df["subject"].str.strip()
 
#---> For each subject column that is missing or empty replace values with "",
#The lambda in python is just a short function without a name, but I named it "s",
#“If the subject still has something in it (after removing spaces), then keep it.
#But if it's empty (like it is blank or just spaces), then write 'No Subject' instead.”
df["subject"] = df["subject"].fillna("").apply(lambda  s: s if s.strip() else "No Subject")

In [15]:
import re
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to C:\Users\RAEEDAH
[nltk_data]     PC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
def clean_body(text):
    
    #Converting the text to lowercase
    text = text.lower() 
    
    #Removing punctuation using regex library, it removes anything not a word or space
    #re.sub() is a function from the re module which handles regex in python.
    #.sub stands for substitue(find and replace), it is taking in 3 arguments:
    #the pattern -> what you are searching for,
    #the replacement -> what you want to put in place 
    #the origial_text -> the text you are searching through
    text = re.sub(r"[^a-zA-Z0-9]", "", text)
    
    #Splits the text into a list of words,
    #we are splitting into words do we can accesss or identify stop words.
    words = text.split()
    
    #Remove stopwords like "the", "is", "at", "an", "this" e.t.c
    #stopwords.words gives a list of English stopwords from NLTK
    #.set -> Converts that list into a set (faster for checking if a word is in the list)
    stop_words = set(stopwords.words("english"))

    #Remove stopword i.e. Go through each word, and only keep it if it’s not a stopword.
    words = [word for word in words if word not in stop_words]
    
    #so this words do not have punctuation or stopwords anymore
    #Join words back into one string from the list
    return " ".join(words)

In [17]:
#Creating a new column clean body that stores the cleaned body now
df["clean_body"] = df["body"].apply(clean_body)

In [18]:
df["clean_body"].head()

0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              forecast
1    traveling business meeting takes fun trip especially prepare presentation would suggest holding business plan meetings take trip without formal business meetings would even try get honest opinions whether trip even desired necessaryas far business meetings think would productive try stimulate discussions across different groups working often presenter speaks others quiet waiting turn meetings might better held round table discussion format suggestion go a

In [19]:
#Creating emailId for easy referncing starting from 1
df["email_id"] = df.index + 1

In [20]:
df.head()

,file,message,subject,body,clean_body,email_id
0,allen-p/_sent_mail/1.,"Message-ID: <18782981.1075855378110.JavaMail.evans@thyme>\nDate: Mon, 14 May 2001 16:39:00 -0700 (PDT)\nFrom: phillip.allen@enron.com\nTo: tim.belden@enron.com\nSubject: \nMime-Version: 1.0\nContent-Type: text/plain; charset=us-ascii\nContent-Transfer-Encoding: 7bit\nX-From: Phillip K Allen\nX-To: Tim Belden <Tim Belden/Enron@EnronXGate>\nX-cc: \nX-bcc: \nX-Folder: \Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Sent Mail\nX-Origin: Allen-P\nX-FileName: pallen (Non-Privileged).pst\n\nHere is our forecast\n\n",No Subject,Here is our forecast,forecast,1
1,allen-p/_sent_mail/10.,"Message-ID: <15464986.1075855378456.JavaMail.evans@thyme>\nDate: Fri, 4 May 2001 13:51:00 -0700 (PDT)\nFrom: phillip.allen@enron.com\nTo: john.lavorato@enron.com\nSubject: Re:\nMime-Version: 1.0\nContent-Type: text/plain; charset=us-ascii\nContent-Transfer-Encoding: 7bit\nX-From: Phillip K Allen\nX-To: John J Lavorato <John J Lavorato/ENRON@enronXgate@ENRON>\nX-cc: \nX-bcc: \nX-Folder: \Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Sent Mail\nX-Origin: Allen-P\nX-FileName: pallen (Non-Privileged).pst\n\nTraveling to have a business meeting takes the fun out of the trip. Especially if you have to prepare a presentation. I would suggest holding the business plan meetings here then take a trip without any formal business meetings. I would even try and get some honest opinions on whether a trip is even desired or necessary.\n\nAs far as the business meetings, I think it would be more productive to try and stimulate discussions across the different groups about what is working and what is not. Too often the presenter speaks and the others are quiet just waiting for their turn. The meetings might be better if held in a round table discussion format. \n\nMy suggestion for where to go is Austin. Play golf and rent a ski boat and jet ski's. Flying somewhere takes too much time.\n",Re:,"Traveling to have a business meeting takes the fun out of the trip. Especially if you have to prepare a presentation. I would suggest holding the business plan meetings here then take a trip without any formal business meetings. I would even try and get some honest opinions on whether a trip is even desired or necessary.\n\nAs far as the business meetings, I think it would be more productive to try and stimulate discussions across the different groups about what is working and what is not. Too often the presenter speaks and the others are quiet just waiting for their turn. The meetings might be better if held in a round table discussion format. \n\nMy suggestion for where to go is Austin. Play golf and rent a ski boat and jet ski's. Flying somewhere takes too much time.",traveling business meeting takes fun trip especially prepare presentation would suggest holding business plan meetings take trip without formal business meetings would even try get honest opinions whether trip even desired necessaryas far business meetings think would productive try stimulate discussions across different groups working often presenter speaks others quiet waiting turn meetings might better held round table discussion format suggestion go austin play golf rent ski boat jet skis flying somewhere takes much time,2
2,allen-p/_sent_mail/100.,"Message-ID: <24216240.1075855687451.JavaMail.evans@thyme>\nDate: Wed, 18 Oct 2000 03:00:00 -0700 (PDT)\nFrom: phillip.allen@enron.com\nTo: leah.arsdall@enron.com\nSubject: Re: test\nMime-Version: 1.0\nContent-Type: text/plain; charset=us-ascii\nContent-Transfer-Encoding: 7bit\nX-From: Phillip K Allen\nX-To: Leah Van Arsdall\nX-cc: \nX-bcc: \nX-Folder: \Phillip_Allen_Dec2000\Notes Folders\'sent mail\nX-Origin: Allen-P\nX-FileName: pallen.nsf\n\ntest successful. way to go!!!",Re: test,test successful. way to go!!!,test successful way go,3
3,allen-p/_sent_mail/1000.,"Message-ID: <13505866.1075863688222.JavaMail.evans@thyme>\nDate: Mon, 23 Oct 2000 06:13:00 -0700 (PDT)\nFrom: phillip.allen@enron.com\nTo: r